<h1><b>Game Recommendation Chatbot: Frank</b></h1>

This application allows the user to interact in a natural tone about their interests in particular video games. The goal of the app is to extract the user's sentiment from their naturally formatted messages and use this to suitably recommend games based on these sentiments.The bot will parse text for any patterns that may indicate the player's preferred console and also the genre of games they prefer. On top of this, it will look out for any sort of descriptive keywords that might give some indication as to just exactly what the user is looking for. 

The bot uses the IGDB api to grab games along with some details from a database. These are grabbed based on platform and genre, the bot then scans through the games associated data (summary, keywords, themes) to scan for some of the keywords that were mentioned by the user. The more keywords a game matches, the higher the game will be sorted in a list. The bot will then recommend from this list. The rest of the games that did not match any keywords but do still match genre and platformer are then sorted by a ‘rating’ value also supplied by the api. 

This is all done in a very free flowing state. I wanted to avoid scenarios where a bot would require the player to respond to a set of questions one at a time or scenarios where the bot would specify how the user should format their messages exactly as I felt this would go against the nature of opting to design a chatbot for this sort of thing as it turns into a standard search bar that is harder to navigate. 

I made sure that any sort of loops or moments where the bot asks a question aren't a possible way that could result in the bot being stuck in a loop. For example if the bot is iterating through game recommendations and the user decides they want to refine their query or even just tell the bot their name, they can do so and the bot will react accordingly.


Three different scenarios of chatbot behaviour

The user may ask for platformers on the Nintendo Switch but then suddenly decide to tell the chatbot they have a XBOX mid way through the bot listing off recommendations. The bot would react to this by confirming with the user if it should update its memory to know the user’s preferred platform is XBOX. When prompted for games again it will suggest platformers (stored at the beginning) on the XBOX (stored just now).

The user may say “My name is Fred” and then go through a bunch of different game recommendations then suddenly say something along the lines of “Actually, call me walter” and the bot will react by confirming if it should stop referring to the user as “Fred” and start using “Walter”.

The user may ask for racing games on the PlayStation one (with no extra specific keywords) and the bot would go through a bunch of random racing games (probably need for speeds), the user may get tired of iterating through and decide to refine their search by saying something like “I want a KART racer with items and multiplayer”, the bot would store this and begin its scan again, this time suggesting a game like Crash Team Racing right away.

I utilized a lot of the material taught in lecture to hit different functionality of the bot. Such as the stuff we learned in week 4 & 5 about the natural language toolkit. I used this to tell the bot how to pick up any keywords that may be of use when refining the search for games. Also, the stuff taught earlier on such as regexes and just generally making use of jupyter notebooks was of great help. I used regexes to extract the most likely intent of the users messages and the use of Jupyter notebooks allowed me to extract and segregate particular parts of my code to iterate over and test with ease. This was especially vital when I was learning what I could do with the IGDB api and what it looks like to call it and what exactly is being sent back to my code.

I think the main loop and premise of my bot is fairly advanced and a little ambitious for the project especially given that I had to do a bulk of the work in a short amount of time due to other responsibilities and coursework.

In [ ]:
from random import Random
import re
import json
import nltk
import requests
from requests.structures import CaseInsensitiveDict
from datetime import date 
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
import time

class ReturnToInteractionLoop(Exception):
    pass

#API AND DATA LOADING FUNCTIONS
def initialize_api():
    global access_token, headers, url
    get_access_token = requests.post('https://id.twitch.tv/oauth2/token?client_id=ch28sul1esy1ul6ye36on4r49f6qxr&client_secret=b0yv81ftmbt2isestd8slt4n7qfd5r&grant_type=client_credentials')
    access_token = get_access_token.json()['access_token']
    
    headers = CaseInsensitiveDict()
    headers["Client-ID"] = "ch28sul1esy1ul6ye36on4r49f6qxr"
    headers["Authorization"] = "Bearer " + access_token
    headers["grant_type"] = "client_credentials"
    headers["Content-Type"] = "application/json"
    
    url = "https://api.igdb.com/v4/games"
def call_api(user_platform, user_genre):
    body = f'''
    fields name, genres.name, platforms.name, release_dates.date, rating, summary, keywords.name, themes.name;
    where platforms.name = "{user_platform}" & genres.name = "{user_genre}";
    limit 500;
    '''
    response = requests.post(url, headers=headers, data=body.encode('utf-8'))
    return response.json()
def load_json():
    with open("chatbot.json") as file:
        data = json.load(file)
    return data

#INTENT MATCHING FUNCTIONS
def match_pattern_to_intent(user_input):
    intent = None
    for pattern_to_intent in data["pattern_to_intent"]:
        if re.search(pattern_to_intent["pattern"], user_input):
            intent = pattern_to_intent["intent"]
            break
    return intent
def match_intent_to_pattern(intent):
    pattern = None
    for pattern_to_intent in data["pattern_to_intent"]:
        if pattern_to_intent["intent"] == intent:
            pattern = pattern_to_intent["pattern"]
            break
    return pattern
def match_intent_to_response(intent):   
    response = None
    if intent in data["intent_to_response"]:
        responseList = data["intent_to_response"][intent]
        response = responseList[Random().randint(0, len(responseList) - 1)]
    return response
def match_intent_to_action(intent):
    action = None
    if intent in data["intent_to_action"]:
        action = data["intent_to_action"][intent]
    if action and action in globals():
        return action
    return action
def get_user_input():
    user_input = input("You: ").lower()
    global last_user_input
    last_user_input = user_input
    return user_input

#INTENT FUNCTIONS
def remember_name(input):
    user_name_re = match_intent_to_pattern("user_name")
    user_name = input[re.search(user_name_re, input).end():].split()[0]
    if memories.get("user_name") and memories["user_name"].lower() != user_name:
        print(f"Frank: Hold up, I thought your name was {memories['user_name']}, would you like to change this to {user_name.capitalize()}?")
        match match_pattern_to_intent(get_user_input()):
            case "positive":
                memories.update({"user_name": user_name.capitalize()})
                return True
            case "negative":
                print(f"Frank: Okay, I'll stick with {memories["user_name"]}.")
                return False
            case _:
                raise ReturnToInteractionLoop()
    memories.update({"user_name": user_name.capitalize()})
    return True
def remember_details(user_input):
    remember_detail(user_input, data["pattern_to_platform"], "platform")
    remember_detail(user_input, data["pattern_to_genre"], "genre")
    remember_keywords(user_input)

    response = "Frank: Right, so"
    if memories.get("platform"):
        response += f" you're on {memories["platform"]},"
    if memories.get("genre"):
        response += f" you're into {memories["genre"]} games,"
    if memories.get("keywords"):
        response += f" you're partial to the following: {", ".join(memories["keywords"]).capitalize()}."
        
    if memories.get("platform") or memories.get("genre") or memories.get("keywords"):
        response = response[:-1] + ". Would you like to hear some recommendations?"
        print(response)
        match(match_pattern_to_intent(get_user_input())):
            case "positive":
                recommend_game(None)
            case "negative":
                print("Frank: Okay, let me know if you change your mind.")
                return False
            case _:
                raise ReturnToInteractionLoop()
    return True
def remember_keywords(user_input):
    custom_stopwords = set(nltk.corpus.stopwords.words('english')).union({"game", "games", "like", "something", "please"})
    user_input = remove_all_patterns_from_string(user_input)
    tokens = nltk.word_tokenize(user_input)
    tagged = nltk.pos_tag(tokens)
    keywords = [word for word, pos in tagged if pos in ['JJ', 'JJR', 'JJS', 'NN', 'NNP', 'NNS', 'VB', 'VBG'] and word not in custom_stopwords]
    keywords.sort()
    if keywords is None: 
        return False
    if len(keywords) == 0:
        return False
    if memories.get("keywords") and memories["keywords"] != keywords:
        print(f"Frank: I had you down for games with the following vibes: {", ".join(memories["keywords"]) + ".".capitalize()} Should I update this to {", ".join(keywords).capitalize()}?")
        match match_pattern_to_intent(get_user_input()):
            case "positive":
                memories.update({"keywords": keywords})
                return True
            case "negative":
                print("Frank: Okay, I'll stick with what I know.")
                return False
            case _:
                raise ReturnToInteractionLoop()
    memories.update({"keywords": keywords})
    return True
def remember_detail(user_input, patterns, memory_key):
    intent = None
    for pattern_to_intent in patterns:
        if re.search(pattern_to_intent["pattern"], user_input):
            intent = pattern_to_intent["intent"]
            break
    if intent is None:
        return False
    if memories.get(memory_key) and memories[memory_key] != intent:
        print(f"Frank: Hold up, I thought you were into {memories[memory_key]} games, would you like to change this to {intent}?")
        match match_pattern_to_intent(get_user_input()):
            case "positive":
                memories.update({memory_key:intent})
                return True
            case "negative":
                print(f"Frank: Okay, I'll stick with {memories[memory_key]}.")
                return False
            case _:
                raise ReturnToInteractionLoop()
    memories.update({memory_key:intent})
    return True
def recommend_game(user_input):
    if user_input is not None:
        remember_detail(user_input, data["pattern_to_platform"], "platform")
        remember_detail(user_input, data["pattern_to_genre"], "genre")
        remember_keywords(user_input)
    if not memories.get("platform"):
        print("Frank: I'm sorry, I don't know what platform you're on. Can you tell me?")
        remember_detail(get_user_input(), data["pattern_to_platform"], "platform")
    if not memories.get("genre"):
        print("Frank: I'm sorry, I don't know what genre you're into. Can you tell me?")
        remember_detail(get_user_input(), data["pattern_to_genre"], "genre")
    


    user_keywords = memories.get("keywords")
    response = call_api(memories['platform'], memories['genre'])
    response = [game for game in response if game['name'] not in memories.get('recommended_games', set())]
    
    
    for game in response:
        game.update({'keyword_count': get_keyword_count_for_game(game, user_keywords)})

    sorted_games = sort_games(response)

    print("Frank: I've got some ideas. Let me know if you want to hear more about any of them.")

    response_list = data["intent_to_response"]["recommend"]
    for game in sorted_games:
        remember_previously_recommended_title(game['name'])
        print("Frank: " + response_list[Random().randint(0, len(response_list) - 1)].format(name=game['name']))
        match match_pattern_to_intent(get_user_input()):
            case "info":
                describe_game(game)
                time.sleep(1)
                print("\nFrank: Still need more recommendations?")
                if match_pattern_to_intent(get_user_input()) == "positive":
                    continue
                else:
                    print("Frank: Okay, let me know if you change your mind.")
                    return False
            case "negative":
                continue
            case "positive":
                print("Frank: Great! I hope you enjoy it!")
                break
            case _:
                raise ReturnToInteractionLoop()
    return False
def sort_games(games):
    games_with_keywords = [game for game in games if 'keyword_count' in game]
    sorted_games_with_keywords = sorted(games_with_keywords, key=lambda x: x['keyword_count'], reverse=True)

    games_without_keywords = [game for game in games if 'keyword_count' not in game]
    sorted_games_without_keywords = sorted(games_without_keywords, key=lambda x: x.get('rating', 0), reverse=True)

    return sorted_games_with_keywords + sorted_games_without_keywords 
def get_keyword_count_for_game(game, keywords):
    keyword_counter = 0
    game_word_set = set()
    if keywords is None:    
        return 0
    if len(keywords) == 0:
        return 0

    if 'keywords' in game:
        for keyword in game['keywords']:
            for word in keyword['name'].split():
                game_word_set.add(word)
    if 'summary' in game:
        game_word_set.add(game['summary'])
    if 'themes' in game:
        for theme in game['themes']:
            for word in theme['name'].split(): 
                game_word_set.add(word)

    for keyword in keywords:
        if keyword in game_word_set:
            keyword_counter += 1
    return keyword_counter
def remember_previously_recommended_title(game_name):
    if 'recommended_games' not in memories:
        memories['recommended_games'] = set()
    if game_name not in memories['recommended_games']:
        memories['recommended_games'].add(game_name)
def describe_game(game):
    result = game['name']
    if 'release_dates' in game:
        if 'date' in game['release_dates'][0]:
            result += " ("+str(date.fromtimestamp(game['release_dates'][0]['date']).year)+")"
    if 'summary' in game:
        result += ": " + game['summary']    
    print("Frank: " + result)
def remove_all_patterns_from_string(input):
    for pattern in data["pattern_to_intent"] + data["pattern_to_platform"] + data["pattern_to_genre"]:
        input = re.sub(pattern["pattern"], "", input)
    return input

#INTERACTION LOOP
def interaction_loop():
    global running
    print("Frank: Hello! My name is Frank! How can I help?")
    use_last_user_input = False
    while running:
        try:
            while True:
                if use_last_user_input:
                    user_input = last_user_input
                    use_last_user_input = False
                else:
                    user_input = get_user_input()
                intent = match_pattern_to_intent(user_input)
                action = match_intent_to_action(intent)
                print_response_after_action = True
                if action is not None:
                    print_response_after_action = globals()[action](user_input)
                if intent is not None and print_response_after_action:
                    response = match_intent_to_response(intent)
                    if response is not None:
                        print("Frank: " + response.format(**memories)) 
                elif action is None and intent is None:
                    print("Frank: I'm sorry, I don't understand that. Can you please rephrase?")
        except ReturnToInteractionLoop:
            use_last_user_input = True
            continue
    
def end_interaction(user_input):
    print("Frank: Goodbye!")
    global running
    running = False
    exit()

#GLOBAL VARIABLES
last_user_input = ""
running = True
memories = {}
data = load_json()
initialize_api()
interaction_loop()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Frank: Hello! My name is Frank! How can I help?


You:  i want an action adventure game on switch with a 3d open-world and dungeons


Frank: I've got some ideas. Let me know if you want to hear more about any of them.
Frank: I recommend Terraria. What do you think?


You:  no


...

You:  i prefer cute games


Frank: I had you down for games with the following vibes: action, dungeons, open-world. Should I update this to Cute?


None
Frank: Hi!


You:  bye


None
Frank: That's too bad.


In [5]:
# ATLEAST 3 INTERESTING RESULTS - AND FUNCTION CALLS TO GENERATE THEM


<h2><b>Example Chat Logs</b></h2> 
<h4>You can copy these prompts and receive the same results</h4>


<h3><u> Example 1 </u> </h3>
Frank: Hello! My name is Frank! How can I help? <br>
You:  howdy <br>
Frank: Hi! <br>
You:  my name is fred <br>
Frank: Nice to meet you!
You:  but you can call me tom <br>
Frank: Hold up, I thought your name was Fred, would you like to change this to Tom? <br>
You:  yes <br>
Frank: Nice to meet you! <br>
You:  i want to play a platformer on my switch <br>
Frank: I've got some ideas. Let me know if you want to hear more about any of them. <br>
Frank: What do you think of Portal?  <br>
You:  no <br>
Frank: I recommend Portal 2. What do you think? <br>
You:  no <br>
Frank: I recommend Super Mario Bros.. What do you think? <br>
You:  actually i want to play on xbox <br>
Frank: Hold up, I thought you were into Nintendo Switch games, would you like to change this to Xbox? <br>
You:  yes <br>
Frank: I've got some ideas. Let me know if you want to hear more about any of them. <br>
Frank: How about Prince of Persia: The Sands of Time? <br>
You:  tell me more <br>
Frank: Prince of Persia: The Sands of Time (2003): Prince of Persia: The Sands of Time is a third-person puzzle-platformer and action-adventure computer and video game published by Ubisoft. It is a reboot of the landmark video game series Prince of Persia created by Jordan Mechner, who served as creative consultant, designer, and scenario writer for The Sands of Time.

Frank: Still need more recommendations? <br>

<h3><u> Example 2 </u> </h3>
Frank: Hello! My name is Frank! How can I help?<br>
You:  Hey there!<br>
Frank: Hello!<br>
You:  My name is Fred<br>
Frank: Hello, Fred!<br>
You:  recommend me some racing games on the ps4 please<br>
Frank: I've got some ideas. Let me know if you want to hear more about any of them.<br>
Frank: How about Star Wars: Episode I - Racer?<br>
You:  no<br>
Frank: How about Grand Theft Auto III?<br>
You:  tell me more<br>
Frank: Grand Theft Auto III (2001): Featuring a fully 3-D living city, a combination of narrative driven and non-linear gameplay and a completely open environment, Grand Theft Auto III represents a huge leap forward in interactive entertainment. For the first time, players are put at the heart of their very own gangster movie, and let loose in a fully-realised 3 dimensional city, in which anything can happen and probably will.

With a cast of hundreds, 50 plus vehicles, ranging from sports cars to ice cream trucks and from boats to buses, 3 hours of music, including opera, reggae, house, drum and bass, pop and disco, a huge array of street ready weapons and some of the seediest characters in video game history, Grand Theft Auto 3 is a sprawling epic which will show you that sometimes, crime can pay and sometimes it can pay you back. Available now for PlayStation2, Xbox, PC and Macintosh.

Frank: Still need more recommendations?<br>
You:  yes<br>
Frank: I recommend Grand Theft Auto: San Andreas. What do you think?<br>
You:  no<br>
Frank: I recommend Grand Theft Auto V. What do you think?<br>
You:  actually call me walter<br>
Frank: Hold up, I thought your name was Fred, would you like to change this to Walter?<br>
You:  yes<br>
Frank: Nice to meet you!<br>



<h2><b>Project Management Reflection</b></h2>

In terms of timeliness, I don't think I'm able to effectively reflect on this due to how the second half of our university term has sort of panned out. Personally, I was assigned 6 different bits of coursework within a very short amount of time (three of which were due at the same time). When I factor in my personal life and responsibilities like starting a new job etc. I did not have much time for this coursework and infact a rather huge portion of this project was done in the final three days before it was actually due.
Sadly I don't believe there was or is much I can really do to control this as naturally life gets in the way of this sort of stuff and paying the rent is a bit more importabt tgab a video game recommendation chatbot.

On a more positive note, due to the short turn around time of this project, it was extremely easy to keep well organized as the project was fresh in my mind at all times. I had a similar approach to the EDA project where I would code away, getting stuff to work and then constantly loop back around and refactor my code to read better and be more maintainable for the future. I feel this is how I work best with projects like this that are fairly new concepts to me that I haven't explored and luckily python compliments this approach well with just how easy it is to get something going with minimal set up.


<h2><b> Process Reflection </b></h2>

As mentioned, this chatbot was put together with a very short development time of essentially three days so there is not much I can really say in terms of weekly development.

I received very valuable feedback from Prashanth the day before submission who suggested some effective ways to help my chatbot's conversation continue to flow naturally and allow the user to feel like they are being forced to respond in a highly rigid format. 

I also got the chance to do some testing with my classmates to see how the bot reacted to their different prompts and preferences. They all had a different style of language when it came to how they interacted with the bot and they also all had a different idea of the kind of games they wanted it to return to them. We found it useful testing to see if we could get the bot to mention a game we had in mind by feeding it enough keywords for example "a ps1 platformer about a bandicoot with collectibles and checkpoints" would return crash 1, crash 2 and then crash 3 before moving onto other games that met less of the criteria.

I used this feedback to go back and reiterate on patterns that match to particular user intentions and also how some of the chatbots responses would pan out after it fulfilled its action to allow for a more natural human like flow of conversation between the user and the bot. 

